In [2]:
#@markdown # Imports
#@markdown ### Bring in many import packages 

from sympy import *
import numpy as np
from tqdm import tqdm
import math
from IPython.display import display, HTML, Javascript

In [5]:
#@markdown # Define HCO Class
#@markdown ## Construct a new HCO object 
#@markdown ### `history: bool`: This is if the historical weights must be saved
class HCO:

  def __init__(self, history = False, weights = None, base = 0):
    """
    Construct a new HCO object
    :param history: This is if the historical must be saved
    """
    if weights is None:
      self.weights = np.random.uniform(-1,1,(8))
    else:
      self.weights = weights
    self.phi = Symbol("phi")
    self.syms = symbols("A:H")
    self.eq = self.generate_eq()
    self.eqs = solve(self.eq,self.phi)
    self.y = 0.25
    self.lr = 1
    self.history = history
    self.historical = []
    self.optim_steps = base
    self.v_curr = 0
    self.s_curr = 0

  # Generate dphi/dt equation see below
  def generate_eq(self):
    eq = 0
    h = lambda n: -n*(n-0.25) * (n-1)
    for i,val in enumerate(self.syms):
      add = ((i % 2) == 1) * 0.5
      neg = (i > 3) * 2 -1
      eq+=h(neg* self.phi + add) * val * neg * -1
    return eq
  
  # Finds zero equation that works
  def ret_eq(self, params):
    vals = []
    out = []
    for eq in self.eqs:
      val = eq.subs(params).n()
      if type(round(val, 10)) == Float:
        vals.append(round(val,10))
        out.append(eq)
    ind = np.abs(np.array(vals)- 0.25).argmin()
    return out[ind], vals[ind]

  # loss comparing neural circuit phi value to 0.25. Also used in backprop.
  def loss(self, yhat):
    return (self.y-yhat)**2

  def loss_prime(self, yhat):
    return (yhat - self.y)*2

  def get_eq(self):
    params = dict(zip(self.syms, self.weights))
    return self.eq.subs(params)

  def print_eq(self):
    params = dict(zip(self.syms, self.weights))
    return self.eq.subs(params).subs({self.phi: Symbol("x")})

  def get_params(self):
    return dict(zip(self.syms, self.weights))

  def backprop_diff(self):
    params = dict(zip(self.syms, self.weights))

    zero = self.ret_eq(params)[1]
    value = 1
  
    while value > 0:
      params = dict(zip(self.syms, self.weights))
      zero = self.ret_eq(params)[1]
      equation = self.eq.diff(self.phi).subs({self.phi: zero})

      value = equation.subs(params).n()
      diff = []
      for sym in self.syms:
        dsym = equation.diff(sym).subs(params).n()
        diff.append(dsym * -1)
      self.weights = self.weights + np.array(diff)

    print("> Done")

  def backprop_optim(self, steps, graph):
    # progress bar
    tq = tqdm(range(steps)) if not graph else range(steps)
    for step in tq:

      params = dict(zip(self.syms, self.weights))
      eq,val = self.ret_eq(params)
      
      # dL/dphi
      loss_grad = round(self.loss_prime(val),10)
      loss = round(self.loss(val),10)

      if loss == 0:
        break

      if graph:
        display(Javascript('addData('+str(step)+','+str(abs(loss))+')'))
      else:
        tq.set_description_str(f"Loss: {abs(loss)}")

      diff = []
      for symbol in self.syms:
        d = round(eq.diff(symbol).subs(params).n(),10)
        # dL/dphi * dphi/dA etc.
        grad = self.adam_optimzer(d * loss_grad)
        diff.append(grad * -1 * self.lr)
      if self.history:
        self.historical.append(self.weights)
      self.weights = self.weights+ np.array(diff)
    return self.historical

  def save_weights(self):
    np.save(f"backprop_weights_{self.optim_steps}.npy", self.weights)
  
  @staticmethod
  def load(path: str, history: bool=False):
    stp = int(path.split("_")[-1])
    return HCO(history, np.load(path,allow_pickle=True), base=stp)
  
  def optim(self, steps, graph = False):
    print("Initializing function for steady states")
    self.backprop_diff()
    print("Starting weights optimization")

    self.backprop_optim(steps, graph)
    self.optim_steps += steps
  def adam_optimzer(self,theta_curr):
    beta1 = 0.9
    beta2 = 0.99
    epsilon = 1e-8
    self.s_curr = beta2 * self.s_curr +(1-beta2)*(np.power(theta_curr, 2))
    self.v_curr = beta1 * self.v_curr + (1-beta1)*theta_curr
    return self.v_curr / math.sqrt(self.s_curr + epsilon)
  def rmsprop(self,theta_curr):
    epsilon = 1e-8
    beta = 0.9
    self.s_curr = beta * self.s_curr +(1-beta)*(np.power(theta_curr, 2))
    return theta_curr / np.sqrt(self.s_curr + epsilon)
  def return_matrices(self):
    gamma = np.zeros((2,2)).tolist()
    alpha = np.zeros_like(gamma).tolist()
    for ind,w in enumerate(self.weights[:4]):
      gamma[ind>1][ind%3 != 0] = w
    for ind,w in enumerate(self.weights[4:]):
      alpha[ind >1][ind%3 != 0] = w
    return alpha, gamma

In [6]:
#@markdown # Run the Backprop Algorithm
#@markdown ### loss should be decreasing.
#@markdown Run this cell if you want to train from scratch.

#@markdown Defines number of optimization iterations
STEPS = 2000 #@param {type:"slider", min:0, max:2000, step:10}
#@markdown Defines if the weights should be saved at every step
HISTORY = False #@param {type:"boolean"}
#@markdown Defines if the realtime loss graph should be plotted. If unchecked it defaults to a progress bar.
GRAPH = False #@param {type:"boolean"}
#@markdown Defines if the weights should be saved to disk for later use.
SAVE = False #@param {type:"boolean"}

hco = HCO(HISTORY)
hco.optim(STEPS, GRAPH)
if SAVE:
  hco.save_weights()
print(hco.return_matrices())

Initializing function for steady states
> Done
Starting weights optimization


Loss: 0.0071775642:  71%|███████   | 1417/2000 [17:21<07:08,  1.36it/s]


KeyboardInterrupt: 

In [1]:
print(hco.return_matrices())

NameError: name 'hco' is not defined

In [ ]:
#@markdown # Load Weights
#@markdown Run this cell if you want to train from a saved weights file. 
#@markdown Check the file explorer on the right for the file name following
#@markdown Follows the following format `backprop_weights_{step number}.npy`

filename = "" #@param {type:"string"}
#@markdown Defines number of optimization iterations
STEPS = 300 #@param {type:"slider", min:0, max:2000, step:10}
#@markdown Defines if the weights should be saved at every step
HISTORY = True #@param {type:"boolean"}
#@markdown Defines if the realtime loss graph should be plotted. If unchecked it defaults to a progress bar.
GRAPH = False #@param {type:"boolean"}
#@markdown Defines if the weights should be saved to disk for later use.
SAVE = False #@param {type:"boolean"}
hco = HCO.load(filename, HISTORY)
hco.optim(STEPS, GRAPH)

if SAVE:
  hco.save_weights()
